In [58]:
import h2o
import librosa
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
from h2o.automl import H2OAutoML

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.111-b14, mixed mode)
  Starting server from C:\Users\JEREMYL\Documents\PitchShift\venv\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\JEREMYL\AppData\Local\Temp\tmpm28_d44z
  JVM stdout: C:\Users\JEREMYL\AppData\Local\Temp\tmpm28_d44z\h2o_JEREMYL_started_from_python.out
  JVM stderr: C:\Users\JEREMYL\AppData\Local\Temp\tmpm28_d44z\h2o_JEREMYL_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Asia/Jerusalem
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.3
H2O_cluster_version_age:,26 days
H2O_cluster_name:,H2O_from_python_JEREMYL_ysg0a4
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.743 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [4]:
data_dir = 'C:\\Users\\JEREMYL\\OneDrive - Technion (1)\\MSc\\Research\\Lung sounds\\data_csv\\'

In [36]:
data_rambam = pd.read_csv(data_dir + 'rambam_mfcc_labels.csv')
data_kaggle = pd.read_csv(data_dir + 'kaggle_mfcc_labels.csv')
data_kauh = pd.read_csv(data_dir + 'KAUH_mfcc_labels.csv')

In [37]:
data_rambam = data_rambam.rename(columns={'gender': 'sex'}).drop(columns=['weight', 'height', 'ID'])

In [38]:
data_kaggle = data_kaggle.drop(columns=['ID', 'bmi'])

In [39]:
data_kauh = data_kauh.drop(columns=['ID'])

In [40]:
data_all = pd.concat([data_rambam, data_kaggle, data_kauh])

In [68]:
print('data_rambam', data_rambam.shape)
print('data_kaggle', data_kaggle.shape)
print('data_kauh', data_kauh.shape)

data_rambam (85, 75)
data_kaggle (918, 75)
data_kauh (336, 75)


In [51]:
X_train, X_test = train_test_split(data_all, test_size=0.2, random_state=42)

In [54]:
X_train.to_csv(data_dir + 'data_train_all_mfcc.csv', index=False)
X_test.to_csv(data_dir + 'data_test_all_mfcc.csv', index=False)

In [55]:
train = h2o.import_file(data_dir + 'data_train_all_mfcc.csv', destination_frame="train_data", header=1)
test = h2o.import_file(data_dir + 'data_test_all_mfcc.csv', destination_frame="test_data", header=1)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [56]:
y = "label"
x = list(train.columns)
x.remove(y)

train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [61]:
aml = H2OAutoML(max_models=1000, max_runtime_secs=60*5, seed=1)
aml.train(x=x, y=y, training_frame=train, validation_frame=test)

AutoML progress: |
16:19:25.235: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.
16:19:25.237: AutoML: XGBoost is not available; skipping it.

███████
16:19:56.160: GLM_1_AutoML_2_20221219_161925 [GLM def_1] failed: java.lang.ArrayIndexOutOfBoundsException

████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_2_AutoML_2_20221219_161925


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    31                 1023                        338262                 1            7            6.87195       2             38            21.5953

ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 0.0027746957031601635
RMSE: 0.05267538042729415
LogLoss: 0.03315517396365061
Mean Per-Class Error: 0.0
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Asthma    Asthma and lung fibrosis    BRON    Bronchiectasis    Bronchiolitis    COPD    COPD + Lung Fibrosis    COPD + Pneumonia    Dyspnea - Pericarditis    Emphysema    Healthy    Heart Failure    Heart Failure + COPD    Heart Failure + Lung Fibrosis     Interstitial Lung Disease    LRTI    Lung Adenocarcinoma    Lung Fibrosis    Mycobacterium Simiae    N     Plueral Effusion    Pneumonia    Pneumonitis    Pulmonary Hypertension + Interstitial Lung Disease    Right Pleural Effusion    Small Cell Lung Cancer    Tracheal Amyloidosis    Tracheal Stenosis    URTI    asthma    copd    heart failure    pneumonia    Error    Rate
--------  --------------------------  ------  ----------------  ---------------  ------  ----------------------  ------------------  ------------------------  -----------  ---------  ---------------  ----------------------  --------------------------------  ---------------------------  ------  ---------------------  ---------------  ----------------------  ----  ------------------  -----------  -------------  ----------------------------------------------------  ------------------------  ------------------------  ----------------------  -------------------  ------  --------  ------  ---------------  -----------  -------  ---------
57.0      0.0                         0.0     0.0               0.0              0.0     0.0                     0.0                 0.0                       0.0          0.0        0.0              0.0                     0.0                               0.0                          0.0     0.0                    0.0              0.0                     0.0   0.0                 0.0          0.0            0.0                                                   0.0                       0.0                       0.0                     0.0                  0.0     0.0       0.0     0.0              0.0          0.0      0 / 57
0.0       3.0                         0.0     0.0               0.0              0.0     0.0                     0.0                 0.0                       0.0          0.0        0.0              0.0                     0.0                               0.0                          0.0     0.0                    0.0              0.0                     0.0   0.0                 0.0          0.0            0.0                                                   0.0                       0.0                       0.0                     0.0                  0.0     0.0       0.0     0.0              0.0          0.0      0 / 3
0.0       0.0                         9.0     0.0               0.0              0.0     0.0                     0.0                 0.0                       0.0          0.0        0.0              0.0                     0.0                               0.0             

In [62]:
y_pred_train = aml.leader.predict(train)[2].as_data_frame().values.flatten()
y_pred_test = aml.leader.predict(test)[2].as_data_frame().values.flatten()

y_train = train[y].as_data_frame().values.flatten()
y_test = test[y].as_data_frame().values.flatten()

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [66]:
y_test

array(['COPD', 'COPD', 'COPD', 'COPD', 'COPD', 'COPD', 'Asthma', 'N',
       'Bronchiolitis', 'COPD', 'COPD', 'COPD', 'Asthma', 'COPD', 'COPD',
       'COPD', 'copd', 'COPD', 'COPD', 'COPD', 'COPD', 'COPD', 'COPD',
       'COPD', 'N', 'COPD', 'COPD', 'N', 'COPD', 'COPD', 'asthma',
       'Pneumonia', 'COPD', 'Lung Fibrosis', 'COPD', 'COPD', 'COPD',
       'COPD', 'COPD', 'COPD', 'COPD with Lung Embolism', 'N', 'COPD',
       'COPD', 'Heart Failure', 'COPD', 'Interstitial Lung Disease',
       'COPD', 'COPD', 'COPD', 'COPD', 'N', 'COPD', 'COPD', 'Healthy',
       'COPD', 'COPD', 'COPD', 'COPD', 'COPD', 'COPD', 'COPD',
       'Plueral Effusion', 'Healthy', 'Asthma', 'Pneumonia', 'COPD',
       'Pneumonia', 'heart failure', 'URTI', 'Bronchiectasis', 'Asthma',
       'N', 'copd', 'COPD', 'COPD', 'COPD', 'COPD', 'COPD', 'N',
       'Lung Fibrosis', 'COPD', 'COPD', 'COPD', 'N', 'COPD', 'COPD',
       'COPD', 'N', 'COPD', 'URTI', 'COPD', 'Asthma', 'COPD', 'Healthy',
       'Healthy', 'COPD', 

In [65]:
y_pred_test

array([9.40311932e-05, 8.40907792e-05, 9.50709386e-05, 2.61988323e-04,
       1.73107686e-04, 1.02344615e-04, 2.94862402e-03, 1.52228724e-04,
       2.84968183e-04, 1.06636100e-04, 9.14573765e-05, 8.76369740e-05,
       2.98510217e-03, 9.39762862e-05, 8.33184636e-05, 4.18793444e-04,
       3.23327666e-03, 1.04895932e-04, 9.43169999e-05, 9.10682330e-05,
       9.54274962e-05, 1.11342352e-04, 8.50218796e-05, 9.17989257e-03,
       5.78710115e-04, 1.36268072e-04, 4.92030290e-04, 1.85021358e-03,
       1.17054286e-04, 8.64611181e-05, 3.30489661e-03, 2.66616465e-04,
       1.02595368e-04, 1.37994348e-02, 8.50276110e-05, 9.17308209e-05,
       2.03858154e-04, 3.11332512e-03, 1.42395523e-03, 7.07331978e-04,
       5.34008310e-03, 1.27413786e-03, 8.49339021e-05, 6.24914000e-05,
       2.60851719e-03, 1.64244423e-04, 6.88697868e-03, 1.11349769e-04,
       2.90036285e-03, 9.02323899e-05, 1.67599117e-04, 1.15026532e-03,
       9.96608644e-05, 5.76752161e-04, 1.03412036e-04, 1.83277617e-04,
      